# CS 4661 Project: Spooky Author Identification (Decision Tree Method)
### By Julie Kasparian

In [87]:
# Importing the required packages and libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [88]:
# Reading test CSV file directly from Web, and store it in a pandas DataFrame:
test_df=pd.read_csv("https://raw.githubusercontent.com/MargaritaNuridj/csula_data_science/master/spooky-author-identification/test.csv")

# Print first 5 rows
test_df.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [89]:
# Reading train CSV file directly from Web, and store it in a pandas DataFrame:
train_df = pd.read_csv('https://raw.githubusercontent.com/MargaritaNuridj/csula_data_science/master/spooky-author-identification/train.csv')

# Print first 5 rows
train_df.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [90]:
# Find unique value counts of each author
train_df['author'].value_counts()

EAP    7900
MWS    6044
HPL    5635
Name: author, dtype: int64

In [91]:
# Split into features and labels
X_train = train_df['text']
y_train = train_df['author']
X_test = test_df['text']

In [92]:
# Transform text data into vectors to run it through machine learning model
vectorizer = CountVectorizer(stop_words='english')
corpus = pd.concat([train_df['text'], test_df['text']])
vectorizer.fit(corpus)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [93]:
# Randomly splitting the original dataset into training set and testing set
X_tr, X_tt, y_tr, y_tt = train_test_split(X_train, y_train, test_size=0.3, random_state=2)

X_tr_new = vectorizer.transform(X_tr)
X_tt_new = vectorizer.transform(X_tt)

In [94]:
# "my_decisiontree" is instantiated as an "object" of DecisionTreeClassifier "class" 
my_decisiontree = DecisionTreeClassifier()
my_decisiontree.fit(X_tr_new, y_tr)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [95]:
# Predicting the Binary Label:
y_predict_dt = my_decisiontree.predict(X_tt_new)
print(y_predict_dt)

['EAP' 'EAP' 'HPL' ... 'EAP' 'HPL' 'EAP']


In [96]:
# Estimating the probability (likelihood) of Each Label: 
y_pred_proba_dt = my_decisiontree.predict_proba(X_tt_new)
y_pred_proba_dt

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [97]:
# Find accuracy score
score_dt = accuracy_score(y_tt, y_predict_dt)
print("Decision tree accuracy:", score_dt)

Decision tree accuracy: 0.5900578821927136


In [98]:
# Write data to CSV file
submission_dt = pd.DataFrame(y_pred_proba_dt, columns=["EAP","HPL","MWS"])
submission_dt['id'] = test_df['id']
submission_dt = submission_dt[["id","EAP","HPL","MWS"]]
submission_dt.to_csv('submission_dt.csv', index=None)